<a href="https://colab.research.google.com/github/gmrwh92/Machine-Learning-Deep-Learning/blob/main/Distilbert_with_1000_sample.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
pip install datasets


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 14.9 MB/s eta 0:00:00


In [3]:
import pandas as pd
import nltk
from nltk.corpus import stopwords, wordnet
from nltk.stem import WordNetLemmatizer
import string
from nltk.tokenize import word_tokenize
import re
from nltk.tag import pos_tag

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import pipeline
from datasets import Dataset
from transformers import TrainingArguments, Trainer
from transformers import EarlyStoppingCallback



In [4]:
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('averaged_perceptron_tagger_eng')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger_eng.zip.


True

In [5]:
## load dataset
df = pd.read_csv('/content/drive/MyDrive/IMDB Dataset.csv')
df['sentiment'] = df['sentiment'].apply(lambda x: 1 if x == 'positive' else 0)
df=df.head(1000)
df

,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1
...,...,...
995,Nothing is sacred. Just ask Ernie Fosselius. T...,1
996,I hated it. I hate self-aware pretentious inan...,0
997,I usually try to be professional and construct...,0
998,If you like me is going to see this in a film ...,0


In [6]:
def get_wordnet_pos(tag):

    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN  # 기본값

In [7]:
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))
def data_preprocessing(text):
    if isinstance(text, str):
        text = text.lower()
        text = re.sub('<.*?>', '', text)  #
        text = ''.join([c for c in text if c not in string.punctuation])

        words = word_tokenize(text)
        words = pos_tag(words)
        words = [lemmatizer.lemmatize(word, get_wordnet_pos(tag)) for word, tag in words if word not in stop_words]

        text = ' '.join(words)
    else:
        text = ''
    return text

df['cleaned_reviews'] = df['review'].apply(data_preprocessing)
df.head()

,review,sentiment,cleaned_reviews
0,One of the other reviewers has mentioned that ...,1,one reviewer mention watch 1 oz episode youll ...
1,A wonderful little production. <br /><br />The...,1,wonderful little production filming technique ...
2,I thought this was a wonderful way to spend ti...,1,think wonderful way spend time hot summer week...
3,Basically there's a family where a little boy ...,0,basically theres family little boy jake think ...
4,"Petter Mattei's ""Love in the Time of Money"" is...",1,petter matteis love time money visually stunni...


In [8]:
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification

In [9]:
model_name = "distilbert-base-uncased"
tokenizer = DistilBertTokenizer.from_pretrained(model_name)
model = DistilBertForSequenceClassification.from_pretrained(model_name)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [10]:
def predict_sentiment(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512)
    with torch.no_grad():
        outputs = model(**inputs)
    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1).item()  # 0 or 1 값 반환
    return  predictions

# 데이터프레임에 적용
df["pretrained"] = df["cleaned_reviews"].apply(predict_sentiment)

In [11]:
df

,review,sentiment,cleaned_reviews,pretrained
0,One of the other reviewers has mentioned that ...,1,one reviewer mention watch 1 oz episode youll ...,1
1,A wonderful little production. <br /><br />The...,1,wonderful little production filming technique ...,1
2,I thought this was a wonderful way to spend ti...,1,think wonderful way spend time hot summer week...,1
3,Basically there's a family where a little boy ...,0,basically theres family little boy jake think ...,1
4,"Petter Mattei's ""Love in the Time of Money"" is...",1,petter matteis love time money visually stunni...,1
...,...,...,...,...
995,Nothing is sacred. Just ask Ernie Fosselius. T...,1,nothing sacred ask ernie fosselius day everybo...,1
996,I hated it. I hate self-aware pretentious inan...,0,hat hate selfaware pretentious inanity masquer...,1
997,I usually try to be professional and construct...,0,usually try professional constructive criticiz...,1
998,If you like me is going to see this in a film ...,0,like go see film history class something like ...,1


# 튜닝

In [12]:
texts = df['cleaned_reviews'].tolist()  # 텍스트
labels = df['sentiment']  # 감성 레이블 (예: 0 = 부정, 1 = 긍정)

In [13]:
dataset = Dataset.from_dict({"text": texts, "label": labels})

In [14]:
train_dataset = dataset.train_test_split(test_size=0.2)['train']
eval_dataset = dataset.train_test_split(test_size=0.2)['test']

In [15]:
def preprocess_function(examples):
    return tokenizer(examples['text'], padding=True, truncation=True)

In [16]:
train_dataset = train_dataset.map(preprocess_function, batched=True)
eval_dataset = eval_dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/800 [00:00<?, ? examples/s]

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

In [17]:
import os
os.environ["WANDB_DISABLED"] = "true"


In [18]:
# 8. 훈련 인자 설정
early_stopping = EarlyStoppingCallback(early_stopping_patience=2)


training_args = TrainingArguments(
    output_dir="./results",           # 결과 저장 디렉터리
    evaluation_strategy="epoch",      # 에포크마다 평가
    learning_rate=3e-5,               # 학습률
    per_device_train_batch_size=16,    # 훈련 배치 사이즈
    per_device_eval_batch_size=16,     # 평가 배치 사이즈
    num_train_epochs=3,               # 에포크 수
    weight_decay=0.01,                # 가중치 감쇠
    logging_dir="./logs",             # 로그 파일 저장 디렉터리
    logging_steps=10,                 # 10 스텝마다 로그 출력
    save_strategy="epoch",            # 모델을 매 에포크마다 저장
    load_best_model_at_end=True,      # 훈련 후 가장 좋은 모델을 로드
)

# 9. Trainer 설정
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    callbacks=[early_stopping]
)

# 10. 모델 훈련
trainer.train()

# 11. 모델 저장
model.save_pretrained("./saved_model")

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Epoch,Training Loss,Validation Loss
1,0.509800,0.432973
2,0.294300,0.269799
3,0.260300,0.235345


In [19]:
# 11. 모델 저장
# 모델과 토크나이저를 저장하는 코드
model.save_pretrained("./my_trained_model")  # 모델 저장
tokenizer.save_pretrained("./my_trained_model")  # 토크나이저 저장


('./my_trained_model/tokenizer_config.json',
 './my_trained_model/special_tokens_map.json',
 './my_trained_model/vocab.txt',
 './my_trained_model/added_tokens.json')

In [20]:
model_path = "./my_trained_model"  # 트레이닝 후 저장된 모델 경로
tokenizer = DistilBertTokenizer.from_pretrained(model_path)
model = DistilBertForSequenceClassification.from_pretrained(model_path)


In [21]:
def predict_sentiment(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512)
    with torch.no_grad():
        outputs = model(**inputs)
    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1).item()  # 0 or 1 값 반환
    return  predictions

# 데이터프레임에 적용
df["fine_tuning"] = df["cleaned_reviews"].apply(predict_sentiment)
df

,review,sentiment,cleaned_reviews,pretrained,fine_tuning
0,One of the other reviewers has mentioned that ...,1,one reviewer mention watch 1 oz episode youll ...,1,1
1,A wonderful little production. <br /><br />The...,1,wonderful little production filming technique ...,1,1
2,I thought this was a wonderful way to spend ti...,1,think wonderful way spend time hot summer week...,1,1
3,Basically there's a family where a little boy ...,0,basically theres family little boy jake think ...,1,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1,petter matteis love time money visually stunni...,1,1
...,...,...,...,...,...
995,Nothing is sacred. Just ask Ernie Fosselius. T...,1,nothing sacred ask ernie fosselius day everybo...,1,0
996,I hated it. I hate self-aware pretentious inan...,0,hat hate selfaware pretentious inanity masquer...,1,0
997,I usually try to be professional and construct...,0,usually try professional constructive criticiz...,1,0
998,If you like me is going to see this in a film ...,0,like go see film history class something like ...,1,0


In [24]:
from sklearn.metrics import accuracy_score

# 정확도 계산
accuracy = accuracy_score(df['pretrained'], df['sentiment'])
print(f"기본 Distilbert-base-uncased 모델 정확도: {accuracy * 100:.2f}%")
accuracy1 = accuracy_score(df['fine_tuning'], df['sentiment'])
print(f"튜닝후 Distilbert-based-uncased 모델 정확도: {accuracy1 * 100:.2f}%")

기본 Distilbert-base-uncased 모델 정확도: 50.10%
튜닝후 Distilbert-based-uncased 모델 정확도: 91.00%
